# 1.0 Data Wrangling <a name="1.1_datawrangling"></a>


## Contents <a name="contents"></a>

* [1 Data Wrangling](#1.1_datawrangling)

    * [Contents](#contents)

* [1.2 Introduction](#1.2_introduction)

    * [1.2.1 Problem Recap](#1.2.1_recap)

    * [1.2.2 Data Wrangling Introduction](#1.2.2_wrangling)
    
* [1.3 Imports](#1.3_imports)

* [1.4 Load Data](#1.4_loaddata)

* [1.5 Explore the Data](#1.5_exploredata)

    * [1.5.1 Fashion data](#1.5.1_fashion)
    
        * [1.5.1.1 Fashion exploration](#1.5.1.1_f_explo)
        
        * [1.5.1.2 Fashion missing data](#1.5.1.2_f_missing)

    * [1.5.2 Electronics data](#1.5.2_electronics)
    
        * [1.5.2.1 Electronics exploration](#1.5.2.1_e_explo)

        * [1.5.2.2 Electronics missing data](#1.5.2.2_e_missing)
        
    * [1.5.3 Home/kitchen data](#1.5.3_h/k)
    
        * [1.5.3.1 Home/kitchen exploration](#1.5.3.1_h/k_e)
        
        * [1.5.3.2 Home/kitchen missing](#1.5.3.2_h/k_e)
            
        

* [1.6 Save Data](#1.6_savedata)<a name="1.6_savedata"></a>
    
    * [1.6.1 Creating a parsing function]
    
    * [1.6.2 Saving the individual files]    
        
        * [1.6.2.1 Fashion CSV]
        
        * [1.6.2.2 Electronics CSV]
        
        * [1.6.2.3 Home/Kitchen CSV]

* [1.7 Summary](#1.7_summary)<a name="1.7_summary"></a>

## 1.1 Data Wrangling 

## 1.2 Introduction<a name="1.2_introduction"></a>
In this notebook, we are going to primarily be focusing on loading the data, examining missing and duplicated values, and dropping unneccesary features. We may do a small amount of data cleaning, but we do not want to prematurely drop features that we may later need.

### 1.2.1 Problem Recap <a name="1.2.1_recap"></a>

Using customer text data about amazon products, we will build, evaluate and compare models to estimate the probability that a given text review can be classified as “positive”, “neutral”, or “negative”.

Our goal is to build a text classifier using Amazon product review data which can be used to analyze customer sentiment which does not have accompanying numeric data.

### 1.2.2 Data Wrangling Introduction <a name="1.2.2_wrangling"></a>
The goal of this notebook will be to read in the data, remove missing values (especially for our input and target features), and then save it in an easier-to-work-with tabular format (CSV). 

## 1.3 imports <a name="1.3_imports"></a>

In [8]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

## 1.4 Load Data<a name="1.4_loaddata"></a>
Datasets are sourced from:

[Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019](https://nijianmo.github.io/amazon/index.html)

The data is split up by product category. 

Our categories will be:

#### Fashion (883,636 reviews)

#### Electronics (20,994,353 reviews)

#### Home and Kitchen (21,928,568 reviews)

The data is available as compressed JSON files. We will use pandas to load the fashion data and determine which features to keep or drop and which features need preliminary cleaning.

Because of how large the Electronics and Home and Kitchen datasets are (5+ GB each), we'll use the fashion dataset to explore the features and build a set of functions to parse each of the three category files. We will stream the JSON data, drop the values/features we do not want to keep, and save to separate CSV files.

## 1.5 Explore the data <a name="1.5_exploredata"></a>

Let's take a look at the top of the **Fashion** dataset

In [43]:
fashion_head = pd.read_json("AMAZON_FASHION.json", lines=True, nrows=200)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
7,3,True,"09 22, 2013",A1BB77SEBQT8VX,B00007GDFV,Darrow H Ankrum II,mother - in - law wanted it as a present for h...,bought as a present,1379808000,NaN,{'Color:': ' Black'},NaN
8,3,True,"07 17, 2013",AHWOW7D1ABO9C,B00007GDFV,rosieO,"Item is of good quality. Looks great, too. But...",Buxton heiress collection,1374019200,NaN,{'Color:': ' Black'},NaN
9,3,True,"04 13, 2013",AKS3GULZE0HFC,B00007GDFV,M. Waltman,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!,1365811200,NaN,{'Color:': ' Black'},NaN
10,4,True,"03 9, 2013",A38NS6NF6WPXS,B00007GDFV,BTDoxies,This brand has been around a long time and you...,BUXTON QUALITY!,1362787200,NaN,{'Color:': ' Black'},NaN
11,2,True,"01 27, 2013",A1KOKO3HTSAI1H,B00007GDFV,Robin Howard,I smoke 100's and these are NOT made for them....,Buxton Heiress Collection Black Leather Cigare...,1359244800,NaN,{'Color:': ' Black'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
175,5,True,"08 28, 2016",A3U8IMV1FKI5Y3,B00008JPRZ,SPB,Love shirt and even makes me look good!,Love the shirt!!!,1472342400,NaN,"{'Size:': ' 16.5 - 34', 'Color:': ' White'}",NaN
176,5,True,"05 25, 2016",A21CTBULJC61N1,B00008JPRZ,Harrycarnival,great shirt,Five Stars,1464134400,NaN,"{'Size:': ' 17.5 - 37', 'Color:': ' White'}",NaN
193,5,False,"03 2, 2006",A3JGLFGCB7KZQX,B0000AWXMM,DG,"It was exactly as described. Beautiful ring, ...",BEAUTIFUL RING!!!! NO REGRETS WHATSOEVER,1141257600,NaN,"{'Size:': ' 12', 'Metal Type:': ' yellow-gold'}",NaN
194,4,False,"08 22, 2005",A179HMPWBPTJ2I,B0000AWXMM,Leana J. Kemp,This is a nice ring. It is simple and elegant...,White Gold Ring,1124668800,6.0,"{'Size:': ' 10', 'Metal Type:': ' white-gold'}",NaN


In [36]:
fashion_head.dtypes

overall             int64
verified             bool
reviewTime         object
reviewerID         object
asin               object
reviewerName       object
reviewText         object
summary            object
unixReviewTime      int64
vote              float64
style              object
dtype: object

The **overall** feature is the number of stars, which we'll be using for our categorical target feature (positive/neutral/negative). It's an integer from 1-5. 

The **verified** feature is a boolean and determines whether Amazon can confirm the user purchased the item they're reviewing (not a free/discounted reviewed item).

The **reviewTime** is in a date string format, and we'll drop it and keep the **unixReviewTime** instead.

The **reviewerID** and **reviewerName** have similar information and we can keep just one (the ID should be fine).

The **asin** is the specific product ID. We could keep it if we want to get item metadata later and use that for additional exploratory analysis (ie. price, product tags, product subcategory).

The **reviewText** will be the main input feature/features we will be using for our sentiment prediction and modelling.

The **summary** is a user provided (or amazon generated) short item summary.

The **vote** column seems to indicate how many users found a review useful.

Let's check the **style** column.

In [47]:
fashion_head[~(fashion_head["style"].isna())]

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
7,3,True,"09 22, 2013",A1BB77SEBQT8VX,B00007GDFV,Darrow H Ankrum II,mother - in - law wanted it as a present for h...,bought as a present,1379808000,NaN,{'Color:': ' Black'},NaN
8,3,True,"07 17, 2013",AHWOW7D1ABO9C,B00007GDFV,rosieO,"Item is of good quality. Looks great, too. But...",Buxton heiress collection,1374019200,NaN,{'Color:': ' Black'},NaN
9,3,True,"04 13, 2013",AKS3GULZE0HFC,B00007GDFV,M. Waltman,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!,1365811200,NaN,{'Color:': ' Black'},NaN
10,4,True,"03 9, 2013",A38NS6NF6WPXS,B00007GDFV,BTDoxies,This brand has been around a long time and you...,BUXTON QUALITY!,1362787200,NaN,{'Color:': ' Black'},NaN
11,2,True,"01 27, 2013",A1KOKO3HTSAI1H,B00007GDFV,Robin Howard,I smoke 100's and these are NOT made for them....,Buxton Heiress Collection Black Leather Cigare...,1359244800,NaN,{'Color:': ' Black'},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
175,5,True,"08 28, 2016",A3U8IMV1FKI5Y3,B00008JPRZ,SPB,Love shirt and even makes me look good!,Love the shirt!!!,1472342400,NaN,"{'Size:': ' 16.5 - 34', 'Color:': ' White'}",NaN
176,5,True,"05 25, 2016",A21CTBULJC61N1,B00008JPRZ,Harrycarnival,great shirt,Five Stars,1464134400,NaN,"{'Size:': ' 17.5 - 37', 'Color:': ' White'}",NaN
193,5,False,"03 2, 2006",A3JGLFGCB7KZQX,B0000AWXMM,DG,"It was exactly as described. Beautiful ring, ...",BEAUTIFUL RING!!!! NO REGRETS WHATSOEVER,1141257600,NaN,"{'Size:': ' 12', 'Metal Type:': ' yellow-gold'}",NaN
194,4,False,"08 22, 2005",A179HMPWBPTJ2I,B0000AWXMM,Leana J. Kemp,This is a nice ring. It is simple and elegant...,White Gold Ring,1124668800,6.0,"{'Size:': ' 10', 'Metal Type:': ' white-gold'}",NaN


The **style** column seems to have sizing and color info but is missing for most reviews.

### 1.5.1 Missing data <a name="1.5.1_missing"></a>

In [49]:
fashion = pd.read_json("AMAZON_FASHION.json", lines=True, chunksize=100000)

Using a json_reader iterator from Pandas, we can iterate through the data rather than loading the entire dataset into memory


In [26]:
missing_stats=[]
for chunk in fashion:
    missing_stats.append(chunk.isna().sum())

In [27]:
missing_vals = pd.DataFrame(missing_stats)

In [30]:
missing_vals

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,0,0,0,0,0,6,109,39,0,90595,28037,98423
1,0,0,0,0,0,18,108,61,0,89490,49765,97805
2,0,0,0,0,0,13,173,68,0,89152,44147,95963
3,0,0,0,0,0,14,172,77,0,86690,41255,94941
4,0,0,0,0,0,6,93,26,0,92429,78710,98412
5,0,0,0,0,0,5,123,49,0,93244,88410,97702
6,0,0,0,0,0,7,174,78,0,92668,87838,96291
7,0,0,0,0,0,11,152,76,0,92517,87214,95966
8,0,0,0,0,0,12,129,59,0,76951,73691,79326


In [31]:
missing_vals.sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          92
reviewText          1233
summary              533
unixReviewTime         0
vote              803736
style             579067
image             854829
dtype: int64

We have no missing values in our **overall** column, which is excellent. The **vote**, **style** and **image** columns have a lot of missing values, which is fine as we'll be dropping those columns. We'll also drop any rows with missing **reviewText**, as that's what we need for our modelling input.

Our final column/feature list to keep will be:

* **overall**
* **reviewerID**
* **verified**
* **asin**
* **reviewText**
* **summary**
* **unixReviewTime**

### 1.5.2 Target Feature <a name="1.5.2_target"></a>

Our target feature will be based on the **overall** column. We will be using the star value for each review to convert the data to Negative/Neutral/Positive categories. This will be the feature that our model trains and predicts with.

In [53]:
cols_to_keep = ["reviewText", "summary", "overall", "verified", "asin", "reviewerID", "unixReviewTime"]

Let's do a quick preview to make sure our column list is correct.

In [54]:
fashion_head[cols_to_keep].head()

,reviewText,summary,overall,verified,asin,reviewerID,unixReviewTime
0,Exactly what I needed.,perfect replacements!!,5,True,7106116521,A1D4G1SNUZWQOT,1413763200
1,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",2,True,7106116521,A3DDWDH9PX2YX2,1411862400
2,Love these... I am going to order another pack...,My New 'Friends' !!,4,False,7106116521,A2MWC41EW7XL15,1408924800
3,too tiny an opening,Two Stars,2,True,7106116521,A2UH2QQ275NV45,1408838400
4,Okay,Three Stars,3,False,7106116521,A89F3LQADZBS5,1406419200


In [55]:
def df_cleaner(temp_df_chunk: pd.DataFrame,
               columns_to_keep: list,
               drop_by_row: list) -> pd.DataFrame:
    
    """This function will take in a chunk of our dataset as a dataframe.
       It will return only the columns we wish to keep and will drop rows
       with missing values in the reviewText column.
       
       Takes:
           temp_df_chunk: a dataframe to be edited
           columns_to_keep: the list of columns we wish to keep from that df       
           drop_by_row: names of cols which will be our subset to drop NAN values
       Returns:
           clean_df: a smaller df with drop_by_row NaNs dropped and only a subset of original columns
       """
    
    
    
    temp_df_chunk[cols_to_keep].dropna(subset="")